In [ ]:
import holoviews as hv
hv.notebook_extension()

In [ ]:
import cmath
import math
import operator
from types import SimpleNamespace
import functools

import kwant
import numpy as np
cot = lambda x: 1 / np.tan(x)

def nu_arg(p):
    num = p.E_z * np.exp(1j * p.theta) - 1j * p.alpha * p.k_x
    denom = p.E_z**2 + p.alpha * p.k_x * (p.alpha * p.k_x - 2 * p.E_z * np.sin(p.theta))
    return num / np.sqrt(denom)

def omega(p, j, sign):
    op = {-1: operator.sub,
          +1: operator.add}[sign]
    return 0.25 * op(gamma(p, j), delta(p, j))

def gamma(p, j):
    k = k_j(p, j)
    x = 1j * k * np.tan(p.W * k / 2)
    return (q(p) + x) / (q(p) - x)

def delta(p, j):
    k = k_j(p, j)
    x = 1j * k * cot(p.W * k / 2)
    return (q(p) - x) / (q(p) + x)

def beta(p, sign):
    w_1 = omega(p, 1, sign)
    w_2 = omega(p, 2, sign)
    nu = nu_arg(p)
    beta = np.array([
        [nu * (w_1 - w_2), (w_1 + w_2)],
        [-(w_1 + w_2), nu.conj() * (w_2 - w_1)]
    ])
    return beta

def S(p):
    if isinstance(p, dict):
        p = SimpleNamespace(**p)
    p.E_z = 0.5 * p.mu_B * p.g * p.B
    r_ll = beta(p, +1)
    t_rl = np.exp(-1j * q(p) * p.W) * beta(p, -1)
    t_lr = np.exp(-1j * q(p) * p.W) * beta(p, -1)
    t_rr = np.exp(-2j * q(p) * p.W) * beta(p, +1)

    S = np.block([[r_ll, t_rl],
                  [t_lr, t_rr]])
    return S

def q(p):
    return np.sqrt((2 * p.m_eff / p.hbar**2) * p.mu_n - p.k_x**2)

def k_j(p, j):
    op = {1: operator.sub,
          2: operator.add}[j]
    p.E_z = 0.5 * p.mu_B * p.g * p.B
    sqrt_term = np.sqrt(
        p.E_z**2
        - 2 * p.alpha * p.E_z * np.sin(p.theta) * p.k_x + p.alpha**2 * p.k_x**2)
    k_sq = (2 * p.m_eff / p.hbar**2) * op(p.mu_n, sqrt_term) - p.k_x**2
    return np.sqrt(k_sq)

In [ ]:
import scipy.constants
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (
        scipy.constants.eV * 1e-3))

params = dict(g=26, mu_n=20, alpha=20, B=1, theta=0, k_x=0.0, W=200, phi=np.pi, **constants)

k_F = lambda p: np.sqrt(p['mu_n'] * (2 * p['m_eff'])) / p['hbar']

## kwant

In [ ]:
@functools.lru_cache()
def make_syst(a=1, W=params['W'], transverse_spin_orbit=True):
    ham = ("(0.5 * hbar**2 * (k_x**2 + k_y**2) / m_eff - mu_n) * sigma_0 + "
           "alpha * (k_y * sigma_x - k_x * sigma_y) + "
           "0.5 * g * mu_B * B * (cos(theta) * sigma_x + sin(theta) * sigma_y)")
    if not transverse_spin_orbit:
        ham = ham.replace('k_y * sigma_x', '0')
    template = kwant.continuum.discretize(ham, grid_spacing=a)
    template_sc = kwant.continuum.discretize(ham.replace('alpha', '0').replace('mu_B', '0'), grid_spacing=a)
    syst = kwant.Builder(kwant.TranslationalSymmetry([a, 0]))
    trs = 1j * np.array([[0, -1j],[1j, 0]])
    lead = kwant.Builder(kwant.TranslationalSymmetry([a, 0], [0, a]), time_reversal=trs)
    syst.fill(template, lambda s: 0 <= s.pos[1] < W, (0, 0))
    syst = kwant.wraparound.wraparound(syst)
    lead.fill(template_sc, lambda s: True, (0, 0))
    lead = kwant.wraparound.wraparound(lead, keep=1)
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    return syst.finalized()
syst = make_syst()

In [ ]:
def energy_operator(syst, params):
    params = dict(params, cos=math.cos, sin=math.sin)
    
    if syst is None:
        smatrix = lambda syst, params: S(params)
    else:
        smatrix = lambda syst, params: kwant.smatrix(syst, params=params).data
    
    if params['k_x'] == 0:
        smat_plus = smat_min = smatrix(syst, params)
    else:
        params['k_x'] *= -1
        smat_min = smatrix(syst, params)
        params['k_x'] *= -1
        smat_plus = smatrix(syst, params)

    phase = cmath.exp(1j * params['phi'] / 2)
    r = np.diag([phase, phase, phase.conjugate(), phase.conjugate()])
    smat_prod = r @ smat_min.conj() @ r.conj() @ smat_plus
    return smat_prod

def energies_over_delta(syst, params):
    """Same as energy_operator(), but returns the
    square-root of the eigenvalues"""
    try:
        from numpy.lib.scimath import sqrt
        operator = energy_operator(syst, params)
        es = -1/np.linalg.eigvals(operator)
        alpha = np.hstack([sqrt(es), -sqrt(es)])
        E = alpha[alpha.imag >= 0].real
        return np.sort(E)
    except Exception as e:
        print(e)
        return np.array(4*[np.nan])

energies_over_delta(syst, params), energies_over_delta(None, params)

# Plot

In [ ]:
from functools import lru_cache
@lru_cache()
def gap_function(ratio, params=params):
    """Returns a function for the gap(k)."""
    syst = make_syst(W=params['W'])
    params['theta'] = np.arctan(4 / ratio)
    k_max = k_F(params)
    ks = np.linspace(0, k_max, 60)
    energies = [energies_over_delta(syst, params) for params['k_x'] in ks]
    E_abs_mins = [np.min(np.abs(energy)) for energy in energies]
    E_gaps = np.minimum.accumulate(E_abs_mins)
    return scipy.interpolate.interp1d(ks, E_gaps)

In [ ]:
ratios = [4, 8, 16]
gap_funcs = {ratio: gap_function(ratio) for ratio in ratios}

In [ ]:
L = np.sqrt((z_x/2)**2 + (2*z_y)**2)
W = 200

from scipy.misc import derivative
def dk_j(k, j, params):
    p = SimpleNamespace(**params)
    def func(k_x):
        p.k_x = k_x
        return k_j(p, j)
    return derivative(func, k, dx=1e-6, order=7)

def dk_12(k, params):
    return abs(dk_j(k, 1, params)), abs(dk_j(k, 2, params))

def dk_12_analytic(k_x, params):
    from cmath import sqrt, sin, cos
    m_eff, hbar, theta, alpha, mu = [params[k] for k in
        ('m_eff', 'hbar', 'theta', 'alpha', 'mu_n')]
    E_z = 0.5 * params['mu_B'] * params['g'] * params['B']
    c = 2 * m_eff / hbar**2
    sub_sqrt = sqrt(E_z**2 - 2 * alpha * E_z * sin(theta) * k_x + alpha**2 * k_x**2)
    k_1 = sqrt(c * (mu - sub_sqrt) - k_x**2)
    k_2 = sqrt(c * (mu + sub_sqrt) - k_x**2)

    num_num = c * (2 * alpha**2 * k_x - 2 * E_z * alpha * sin(theta))
    num_den = 2 * sqrt(alpha**2 * k_x**2 - 2 * E_z * alpha * k_x * sin(theta) + E_z**2)

    num_1 = - num_num / num_den - 2 * k_x
    num_2 = num_num / num_den - 2 * k_x

    den_1 = 2 * k_1
    den_2 = 2 * k_2

    dk1 = num_1 / den_1
    dk2 = num_2 / den_2
    dk1 = dk1.real if abs(dk1.imag) < 1e-15 else np.nan
    dk2 = dk2.real if abs(dk2.imag) < 1e-15 else np.nan
    return dk1, dk2

def check(dk_12):
    dk1, dk2 = dk_12
    return abs(dk1) < W / L and abs(dk2) < W / L

k_max = k_F(params)
ks = np.linspace(-k_max, k_max, 501)
ks_allowed = np.array([check(dk_12_analytic(k, params=params)) for k in ks])

In [ ]:
ks_allowed

In [ ]:
import matplotlib.pyplot as plt

def cutoff_factor(z_x, z_y, W):
    theta = np.arctan(4 * z_y / z_x)
    crossing_point = W / np.cos(theta)
    if z_y < crossing_point / 2:
        return np.nan
    D = np.sqrt(z_x**2 + (2 * z_y + W * np.sqrt(1 + (4 * z_y/z_x)**2))**2)
    return np.sqrt(1 - (W / D)**2)

def k_cutoff(z_y, ratio, params=params):
    z_x = z_y * ratio
    return k_F(params) * cutoff_factor(z_x, z_y, params['W'])

fig, ax = plt.subplots()
z_ys = np.arange(10, 405, 5)
for ratio in ratios:
    gap_func = gap_funcs[ratio]
    gaps = np.array([gap_func(k_cutoff(z_y, ratio)) for z_y in z_ys])
    ax.plot(z_ys, gaps, label=f'ratio {ratio}')
    ax.legend()

ax.set_ylim(0, 0.2)
plt.show()

In [ ]:
f = gap_funcs[8]
params['theta'] = np.arctan(4 / ratio)
cut = k_F(params)
ks = np.linspace(0, cut)
gaps = np.array([f(k) for k in ks])

dm = hv.DynamicMap(lambda z_y, ratio: hv.VLine(
    k_cutoff(z_y, ratio)), kdims=['z_y', 'ratio']).redim.range(z_y=(10, 400), ratio=(4, 16))

hv.Curve((ks, gaps)) * dm

### Paths

In [ ]:
%%opts Path [show_legend=True]
def plot(B, alpha, mu_n, theta, phi, params=params):
    params['B'] = B
    params['alpha'] = alpha
    params['mu_n'] = mu_n
    params['theta'] = theta
    params['phi'] = phi
    syst = make_syst(W=params['W'])
    k_max = 0.92*k_F(params)
    ks = np.linspace(-k_max, k_max, 50)
    Es_ana = [energies_over_delta(None, params) for params['k_x'] in ks]
    Es_num = [energies_over_delta(syst, params) for params['k_x'] in ks]
    return (hv.Path((ks, Es_ana), label='analytics')[:, -1.1:1.1] 
            * hv.Path((ks, Es_num), label='numerics')[:, -1.1:1.1])

hv.DynamicMap(plot, kdims=['B', 'alpha', 'mu_n', 'theta', 'phi']).redim.range(
    B=(0.01, 1), alpha=(1, 100), mu_n=(10, 50), theta=(0, np.pi), phi=(0, np.pi))